In [ ]:
from lvmtipo.actors import lvm

telescope_subsystem_name = "sci"
tss = await lvm.from_string(telescope_subsystem_name).start()

await tss.foc.moveAbsolute(42)
await tss.pwi.status()

In [ ]:
%matplotlib inline

from logging import DEBUG, INFO
from lvmtipo.actors import lvm
from lvmagp.focus import Focus
from lvmagp.images.processors.detection import DaophotSourceDetection, SepSourceDetection

from plotimg import plot_images

source_detection = SepSourceDetection()
focus = Focus(tss, source_detection = source_detection, radius_column = "fwhm", level = DEBUG)

await tss.pwi.status()

def img_cb(imgs):
   plot_images(imgs)

f_cal, f_err = await focus.fine(guess=44, count=2, step=3, exposure_time=10, callback=img_cb)

print(f"(focus, err): {f_cal} {f_err}")


In [ ]:
from plotimg import plot_images
from lvmagp.images import Image

print(f_cal[0])
await tss.foc.moveAbsolute(f_cal[0])
filenames = (await tss.agc.expose(0.5)).flatten().unpack("*.filename") 

plot_images([Image.from_file(f) for f in filenames])
